In [12]:
import copy
import itertools
import math


class ImpossibleTriangleError(Exception):
    def __init__(self, beam_distance):
        self.beam_distance = beam_distance


class Wall:
    def __init__(self, wall, dimensions, my_position, trainer_position, room):
        self.wall = wall
        self.axis = wall % 2
        self.length = dimensions[self.axis]
        self.span = dimensions[not self.axis]
        self.distance_to_me = dimensions[not self.axis] - my_position[not self.axis] if wall < 2 else my_position[not self.axis]
        self.distance_to_trainer = dimensions[not self.axis] - trainer_position[not self.axis] if wall < 2 else trainer_position[not self.axis]
        self.room = room

        def turn(self, direction):
            return self.room.walls[(self.wall + direction) % 4]

        def reflect(self):
            return self.room.walls[(self.wall + 2) % 4]


class Room:
    def __init__(self, walls, dimensions, my_position, trainer_position):
        self.walls = [Wall(wall, dimensions, my_position, trainer_position, self) for wall in walls]
        self.my_position = my_position
        self.trainer_position = trainer_position


class Path:
    def __init__(self, first_wall, second_wall, direction, room):
        self.walls = [first_wall, second_wall]
        self.direction = direction
        self.room = room
        self.axis = first_wall.axis if first_wall.axis == second_wall.axis else None
        self.turned = True if self.axis is None else False
        self.opposite_axis = first_wall % 2
        self.adjacent = first_wall.distance_to_me + first_wall.span
        self.opposite = first_wall.turn().distance_to_me

    def turn(self):
        path = copy.deepcopy(self)
        if not path.turned: path.turned = True
        wall = self.walls[-1].turn()
        path.opposite += wall.length if wall.axis == self.opposite_axis else 0
        path.adjacent += wall.length if wall.axis != self.opposite_axis else 0
        path.walls.append(wall)
        return path
        
    def reflect(self):
        path = copy.deepcopy(self)
        wall = self.walls[-1].reflect()
        if not path.axis: path.axis = wall.axis
        path.direction = -path.direction
        path.opposite += wall.span if wall.axis != path.opposite_axis else 0
        path.adjacent += wall.span if wall.axis == path.opposite_axis else 0
        path.walls.append(wall)
        return path

    def extend(self):
        if self.axis is None or self.walls[-1].axis == self.axis: return (self.reflect(), self.turn())
        else: return (self.turn(),)

    def beam_distance(self):
        wall = self.walls[-1]

        if wall.axis == self.opposite_axis:
            self.adjacent += wall.distance_to_trainer
            self.opposite -= wall.turn().distance_to_trainer

        if wall.axis != self.opposite_axis:
            self.adjacent -= wall.turn().distance_to_trainer
            self.opposite += wall.distance_to_trainer

        if self.adjacent == 0 or self.opposite == 0:
            beam_distance = sum(self.adjacent + self.opposite)
            raise ImpossibleTriangleError(beam_distance)

        first_adjacent = float(wall[0].distance_to_me)
        first_opposite = first_adjacent * self.opposite / self.adjacent
        first_hypotenuse = math.sqrt(first_adjacent ** 2 + first_opposite ** 2)
        beam_distance = first_hypotenuse * self.opposite / first_opposite

        if first_opposite > self.walls[0].turn().distance_to_me:
            raise ImpossibleTriangleError(beam_distance)

        if not self.turned and wall.turn().distance_to_trainer > wall.turn().distance_to_me:
            raise ImpossibleTriangleError(beam_distance)


def solution(dimensions, my_position, trainer_position, distance):

    walls = [0,1,2,3]
    moves = [-1,2,2,1]
    directions = [-1,-1,1,1]
    room = Room(walls, dimensions, my_position, trainer_position)
    stack = [Path(room.walls[wall], room.walls[(wall + move) % 4], direction, room) for wall in walls for move, direction in zip(moves, directions)]

    if math.sqrt((trainer_position[0] - my_position[0]) ** 2 + (trainer_position[1] - my_position[1]) ** 2) <= distance: result = 1
    else: return 0

    for wall in room.walls:
        opposite_axis = wall.wall % 2
        first_adjacent = float(wall.distance_to_me)
        adjacent = first_adjacent + wall.distance_to_trainer
        opposite = abs(trainer_position[opposite_axis] - my_position[opposite_axis])
        first_opposite = first_adjacent * opposite / adjacent
        first_hypotenuse = math.sqrt(first_adjacent ** 2 + first_opposite ** 2)
        beam_distance = first_hypotenuse * opposite / first_opposite
        
        if beam_distance < distance:
            result += 1

    # write base cases

    while stack:
        path = stack.pop()

        try:
            if path.beam_distance() <= distance:
                result += 1
                stack.extend(path.extend())

        except ImpossibleTriangleError as e:
            if e.beam_distance < distance:
                stack.extend(path.extend())

AttributeError: 'int' object has no attribute 'distance_to_me'

In [14]:
# solution([3,2], [1,1], [2,1], 4)

In [13]:
# paths = solution([300,275], [150,150], [185,100], 500)
for path in paths: print([wall.wall for wall in path.walls], path.direction)

[0, 3] -1
[0, 2] -1
[0, 2] 1
[0, 1] 1
[1, 0] -1
[1, 3] -1
[1, 3] 1
[1, 2] 1
[2, 1] -1
[2, 0] -1
[2, 0] 1
[2, 3] 1
[3, 2] -1
[3, 1] -1
[3, 1] 1
[3, 0] 1
